In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
import pandas as pd
from PIL import ImageDraw, ImageFont, Image
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
import torch.nn as nn
from transformers import ViTModel
from torchinfo import summary  # 
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import random
import time

In [2]:
DEVICE="cuda:0"
def setAllSeeds(seed):
  os.environ['MY_GLOBAL_SEED'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
setAllSeeds(42)

In [3]:
df = pd.read_csv("train.csv")
categories=df["Category"].unique()
print(categories)
category=categories[4]
df = df[df["Category"]==category]
test_df = pd.read_csv("test.csv")
test_df = test_df[test_df["Category"]==category]
test_df.head()

['Men Tshirts' 'Sarees' 'Kurtis' 'Women Tshirts' 'Women Tops & Tunics']


,id,Category
23280,23564,Women Tops & Tunics
23281,23565,Women Tops & Tunics
23282,23566,Women Tops & Tunics
23283,23567,Women Tops & Tunics
23284,23568,Women Tops & Tunics


In [4]:
id2label={}
label2id={}
attrs={}
total_attr=len(df.columns)
for i in range(3,total_attr):
    labels=df[df.columns[i]].unique()
    id2label[i-3]={k:labels[k] for k in range(len(labels))}
    label2id[i-3]={labels[k]:k for k in range(len(labels))}
    attrs[i-3]=df.columns[i]
print(id2label)
print(label2id)
print(attrs)

{0: {0: 'black', 1: 'navy blue', 2: 'red', 3: 'default', 4: 'maroon', 5: 'white', 6: nan, 7: 'green', 8: 'blue', 9: 'pink', 10: 'yellow', 11: 'peach', 12: 'multicolor'}, 1: {0: 'regular', 1: 'fitted', 2: 'boxy', 3: nan, 4: 'default'}, 2: {0: nan, 1: 'crop', 2: 'regular'}, 3: {0: nan, 1: 'high', 2: 'round neck', 3: 'stylised', 4: 'sweetheart neck', 5: 'v-neck', 6: 'square neck', 7: 'default'}, 4: {0: nan, 1: 'casual', 2: 'party'}, 5: {0: nan, 1: 'default', 2: 'printed', 3: 'solid'}, 6: {0: nan, 1: 'solid', 2: 'typography', 3: 'graphic', 4: 'default', 5: 'quirky', 6: 'floral'}, 7: {0: nan, 1: 'short sleeves', 2: 'sleeveless', 3: 'three-quarter sleeves', 4: 'long sleeves'}, 8: {0: 'regular sleeves', 1: 'default', 2: 'sleeveless', 3: nan, 4: 'puff sleeves'}, 9: {0: nan, 1: 'knitted', 2: 'default', 3: 'ruffles', 4: 'waist tie-ups', 5: 'tie-ups', 6: 'applique'}}
{0: {'black': 0, 'navy blue': 1, 'red': 2, 'default': 3, 'maroon': 4, 'white': 5, nan: 6, 'green': 7, 'blue': 8, 'pink': 9, 'yellow

In [6]:
from transformers import ViTImageProcessor
model_name = f'vit/{category}/final'
processor = ViTImageProcessor.from_pretrained(model_name)

In [7]:
class CustomFashionManager(Dataset):
    def __init__(self,csv_file, root_dir="./",transforms =None):
        self.fashionItems = csv_file
        self.root_dir = root_dir
        self.transforms = transforms
    
    def __len__(self):
        return len(self.fashionItems)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,f"{self.fashionItems.iloc[idx, 0]:06d}"+'.jpg')
        image = Image.open(img_name)
        inputs=processor(image, return_tensors='pt')
        

        # if self.transforms:
        #     sample = self.transforms(sample)
        return inputs


In [8]:
train_fashion_data = CustomFashionManager(csv_file=df,root_dir='train_images')
test_fashion_data = CustomFashionManager(csv_file=test_df,root_dir='test_images')
        

<Figure size 640x480 with 0 Axes>

In [9]:
import sys
from typing import List
from transformers import ViTConfig,ViTPreTrainedModel


class CustomConfig(ViTConfig):
    def __init__(self,num_classes_per_label:List[int]=[1],**kwargs):
        super().__init__(**kwargs)
        self.num_classes_per_label = num_classes_per_label

class MultiLabelMultiClassViT(ViTPreTrainedModel):
    config_class=CustomConfig
    def __init__(self, config: CustomConfig) -> None:
        super().__init__(config)

        self.vit = ViTModel(config, add_pooling_layer=False)
        self.classifiers = nn.ModuleList([
            nn.Linear(config.hidden_size, num_classes) 
            for num_classes in config.num_classes_per_label
        ])
        # Initialize weights and apply final processing
        self.post_init()

    def forward(self, pixel_values,labels=None):
        outputs = self.vit(pixel_values).last_hidden_state[:, 0, :]  # CLS token representation
        logits = [classifier(outputs) for classifier in self.classifiers]
        if labels is not None:
            loss=0
            for i in range(len(logits)):
                target=labels[:,i]
                loss += torch.nn.functional.cross_entropy(logits[i], target)
            return {"loss": loss, "logits": logits}
        return {"logits": logits}



In [10]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import classification_report

model = MultiLabelMultiClassViT.from_pretrained(model_name)

def collate_fn(batch):
    return {
        'pixel_values': torch.cat([x['pixel_values'] for x in batch],dim=0),
    }

def compute_metrics(pred):
    logits = pred.predictions
    labels=pred.label_ids
    probs = np.stack([np.argmax(logit,axis=1) for logit in logits])
    probs=probs.T
    report=classification_report(labels.flatten(),probs.flatten(),output_dict=True)
    return {'accuracy': report['accuracy'],"macro avg f1":report['macro avg']['f1-score']}

training_args = TrainingArguments(
  output_dir="./vit/"+category,
  per_device_train_batch_size=96,
  per_device_eval_batch_size=96,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  num_train_epochs=1,
  fp16=True,
  learning_rate=2e-4,
  save_total_limit=1,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='wandb',
  load_best_model_at_end=True,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_fashion_data,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [11]:
y_pred=trainer.predict(test_fashion_data)

[2024-10-16 14:20:33,950] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/aseems/anaconda3/envs/mhcp4/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karan21258 (karan912). Use `wandb login --relogin` to force relogin


In [12]:
logits = y_pred.predictions
probs = np.stack([np.argmax(logit,axis=1) for logit in logits])
probs=probs.T
l=[]
for i in range(len(probs)):
    x=[]
    for j in range(len(probs[i])):
        x.append(id2label[j][probs[i][j]])
    l.append(x)
test_df['len']=len(l[0])
for i in range(10):
    x=[]
    for j in range(len(l)):
        if i<len(l[0]) and l[j][i]!=np.nan:
            x.append(l[j][i])
        else:
            x.append(np.nan)
    test_df[f"attr_{i+1}"]=x

test_df.to_csv(f"preds/{category}.csv",index=False)

In [13]:
import gc
del model, trainer
torch.cuda.empty_cache()
gc.collect()

2596